# IBM Capstone Project
### by Marc Gou

## Phase 2: Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
---
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:
![alt text][logo]
[logo]: https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1546646400000&hmac=CRqh9ZyRbEZRxrrEhJftbfRQumTwfQDD3wnfdlYgc08
---  
3. To create the above dataframe:

  * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
  * Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
  * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11** in the above table.
  * If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. So for the **9th** cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be **Queen's Park.**
  * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
  * In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.
---    
4. Submit a link to your Notebook on your Github repository. **(10 marks)**

## My Submission:

**Step 1: Installation and import of useful libraries**

In [87]:
# Installation of useful libraries
# !conda install -c conda-forge beautifulsoup4 --yes

# Import of useful libraries
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import urllib.request as req

**Step 2: Request to the wiki page and parse it with BeautifulSoup, and locate the Toronto neighborhoods table**

In [88]:
# Open the URL
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_req = req.urlopen(wiki_url)

# Parsing with BeautifulSoup
wiki_soup = BeautifulSoup(wiki_req, 'lxml')
# print(wiki_soup.prettify())

*We can see that the table of Toronto neighborhoods are under the class "wikitable sortable"*

In [89]:
# Locate the table containing the 
neig_table = wiki_soup.find('table', attrs={'class':'wikitable sortable'})

**Step 3: Convert the XML Tree above to a dataframe**

*We can see that the headers are marked with the tag __th__*

In [90]:
# Extract all the headers
table_h = neig_table.findAll('th')

# Define the headers
headers = []
for h in table_h:
    headers.append(h.contents[0].replace('\n',''))

*We can see that each neighborhood are marked with the tag __tr__ with __td__ for the value of their attributes*

In [91]:
# Extract all the neighborhood data
table_n = neig_table.findAll('td')

# Insert the data, 3 elements for each postal code
full_data = []
for i in range(0, len(table_n)-3, 3):
    pc_data = []
    e1 = table_n[i].contents[0]
    e2 = table_n[i+1].contents[0]
    e3 = table_n[i+2].contents[0]
    pc_data.append(e1.replace('\n',''))
    if(str(e2)[0:2] == '<a'):
        pc_data.append(e2.get('title'))
    else:
        pc_data.append(e2.replace('\n',''))    
    if(str(e3)[0:2] == '<a'):
        pc_data.append(e3.get('title'))
    else:
        pc_data.append(e3.replace('\n',''))
    full_data.append(pc_data)

*Now let's convert the data to a dataframe*

In [92]:
df = pd.DataFrame(data = full_data, columns = headers)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)


**Step 4: Process the dataframe according to the requirements**

In [99]:
# Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df.Borough

# When more than one neighborhood can exist in one postal code area: these two rows will be combined into one row 
# with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

*In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe*

In [100]:
df.shape

(103, 3)

**Step 5: Process the dataframe according to the requirements**

*Download the geospatial data, save it as a CSV file and read it into a pandas dataframe*

In [101]:
!wget -q -O 'toronto_geodata.csv' http://cocl.us/Geospatial_data
geo_df = pd.read_csv('toronto_geodata.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [140]:
toronto_merged = df
toronto_merged = toronto_merged.join(geo_df.set_index('Postal Code'), on='Postcode')
toronto_merged.shape

(103, 5)

**Step 6: Explore and cluster the neighborhoods in Toronto.**

*Note: You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.* 
 
Make sure that:
1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

In [111]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

 # import the library to handle requests
import requests

# install and import map rendering library
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

*Let's create a function to repeat the same process to all the neighborhoods in Manhattan*

In [112]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    CLIENT_ID = 'K4SABZ32C4O5P4H2QVP4AAYZUKZFTLS3WPRN3ZISL24I5TJW' # your Foursquare ID
    CLIENT_SECRET = 'CEL0PP1DNVSIKFJ2HDXPIY2W0UQTGA1UZWRE1JC3YORY5HRY' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            1000)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

*Run the above function on each neighborhood and create a new dataframe called manhattan_venues, and check the size of the resulting dataframe*

In [116]:
toronto_venues = getNearbyVenues(names=toronto_merged['Postcode'],latitudes=toronto_merged['Latitude'],longitudes=toronto_merged['Longitude'])
print('The shape of the dataframe toronto_venues is: {}'.format(toronto_venues.shape))
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.head()

The shape of the dataframe toronto_venues is: (2229, 7)
There are 272 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1B,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,M1C,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


*Create a new data frame based on the venue category and group it by Neighborhood (Postcode)*

In [126]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# Group by Postcode
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
print('The shape of the dataframe toronto_grouped is: {}'.format(toronto_grouped.shape))
toronto_grouped.head()

The shape of the dataframe toronto_grouped is: (99, 273)


,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000

In [141]:
# Function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [142]:
# Create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] =toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Print Shop,Fast Food Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
1,M1C,Bar,Golf Course,Yoga Studio,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,M1E,Rental Car Location,Pizza Place,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Yoga Studio,Drugstore,Dog Run,Doner Restaurant
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
4,M1H,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Caribbean Restaurant,Thai Restaurant,Fried Chicken Joint,Drugstore,Doner Restaurant,Donut Shop
5,M1J,Playground,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
6,M1K,Discount Store,Bus Station,Department Store,Hobby Shop,Coffee Shop,Train Station,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,M1L,Bakery,Bus Line,Intersection,Metro Station,Fast Food Restaurant,Park,Soccer Field,Farmers Market,Falafel Restaurant,Event Space
8,M1M,Motel,American Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
9,M1N,College Stadium,General Entertainment,Skating Rink,Café,Concert Hall,Diner,Farmers Market,Falafel Restaurant,Event Space,Comfort Food Restaurant


*Run k-means to cluster the neighborhood into 5 clusters.*

In [168]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

*Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.*

In [169]:
toronto_final_df = toronto_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_final_df = toronto_final_df.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

# Ignore postcodes without venue data
toronto_final_df = toronto_final_df.dropna().reset_index(drop=True)
toronto_final_df['Cluster Labels'] = toronto_final_df['Cluster Labels'].apply(np.int64)
toronto_final_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M1B,"Scarborough, Toronto","Rouge, Toronto, Malvern, Toronto",43.806686,-79.194353,Print Shop,Fast Food Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,0
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto), Rouge Hill, Port Uni...",43.784535,-79.160497,Bar,Golf Course,Yoga Studio,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,0
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, Toronto, West Hill, To...",43.763573,-79.188711,Rental Car Location,Pizza Place,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,0
3,M1G,"Scarborough, Toronto","Woburn, Toronto",43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,0
4,M1H,"Scarborough, Toronto","Woburn, Toronto",43.773136,-79.239476,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Caribbean Restaurant,Thai Restaurant,Fried Chicken Joint,Drugstore,Doner Restaurant,Donut Shop,0
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476,Playground,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,4
6,M1K,"Scarborough, Toronto","East Birchmount Park, Ionview, Kennedy Park, T...",43.727929,-79.262029,Discount Store,Bus Station,Department Store,Hobby Shop,Coffee Shop,Train Station,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,0
7,M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Toronto, Oakridge, Toronto",43.711112,-79.284577,Bakery,Bus Line,Intersection,Metro Station,Fast Food Restaurant,Park,Soccer Field,Farmers Market,Falafel Restaurant,Event Space,0
8,M1M,"Scarborough, Toronto","Cliffcrest, Cliffside, Toronto, Scarborough Vi...",43.716316,-79.239476,Motel,American Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner,0
9,M1N,"Scarborough, Toronto","Birch Cliff, Cliffside West",43.692657,-79.264848,College Stadium,General Entertainment,Skating Rink,Café,Concert Hall,Diner,Farmers Market,Falafel Restaurant,Event Space,Comfort Food Restaurant,0


*Visualize the resulting clusters*

In [170]:
# Import the libraries for location
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Define address
address = 'Toronto City'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:12: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [171]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final_df['Latitude'], toronto_final_df['Longitude'], toronto_final_df['Postcode'], toronto_final_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters